In [10]:
import sys
import os
from pathlib import Path

import sys
import os
from pathlib import Path
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
sys.path.append("../models")
processed_series_path = Path(parent_dir) / 'processed_series'
import copy
from scipy.special import erf
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
import pickle

print(sys.path)
from ICL import MultiResolutionPDF



['/home/admin-quad/Toni/LLMICL_inPCA/figures_analysis', '/home/admin-quad/anaconda3/envs/LLM/lib/python311.zip', '/home/admin-quad/anaconda3/envs/LLM/lib/python3.11', '/home/admin-quad/anaconda3/envs/LLM/lib/python3.11/lib-dynload', '', '/home/admin-quad/anaconda3/envs/LLM/lib/python3.11/site-packages', '/home/admin-quad/LLM/llama', '/home/admin-quad/Toni/LLMICL_inPCA', '../models', '/home/admin-quad/Toni/LLMICL_inPCA', '../models']


In [11]:
all_files = sorted([path for path in processed_series_path.iterdir()], key=lambda x: x.name)
for index, file in enumerate(all_files):
    series_dict = pickle.load(file.open('rb'))
    llama_size = series_dict['llama_size']
    random_seed = series_dict['random_seed']
    print(f"[{index}]: {file.name}, {llama_size}, {random_seed}")

[0]: uncorrelated_gaussian_centered_sigma_0.1_0.pkl, 13b, 2
[1]: uncorrelated_gaussian_centered_sigma_0.1_1.pkl, 13b, 3
[2]: uncorrelated_gaussian_centered_sigma_0.1_2.pkl, 7b, 2
[3]: uncorrelated_gaussian_centered_sigma_0.1_3.pkl, 7b, 3
[4]: uncorrelated_gaussian_centered_sigma_0.1_4.pkl, 70b, 2
[5]: uncorrelated_gaussian_centered_sigma_0.1_5.pkl, 70b, 3
[6]: uncorrelated_gaussian_centered_sigma_0.1_KDE_0.pkl, KDE, 2
[7]: uncorrelated_gaussian_centered_sigma_0.1_KDE_1.pkl, KDE, 3
[8]: uncorrelated_gaussian_centered_sigma_0.1_histogram_0.pkl, histogram, 2
[9]: uncorrelated_gaussian_centered_sigma_0.1_histogram_1.pkl, histogram, 3
[10]: uncorrelated_gaussian_centered_sigma_0.3_0.pkl, 13b, 2
[11]: uncorrelated_gaussian_centered_sigma_0.3_1.pkl, 13b, 3
[12]: uncorrelated_gaussian_centered_sigma_0.3_2.pkl, 7b, 2
[13]: uncorrelated_gaussian_centered_sigma_0.3_3.pkl, 7b, 3
[14]: uncorrelated_gaussian_centered_sigma_0.3_4.pkl, 70b, 2
[15]: uncorrelated_gaussian_centered_sigma_0.3_5.pkl, 70b, 

In [12]:

alpha = 1



file = all_files[24]
dist_type = 'uniform'

file = all_files[12]

file = all_files[22]

file = all_files[6]

file = all_files[4]

file = all_files[2]

file = all_files[16]

file = all_files[38]

dist_type = 'gaussian'


processed_dict = pickle.load(file.open('rb'))
full_series = processed_dict['full_series']
rescaled_true_mean_arr = processed_dict['rescaled_true_mean_arr']
rescaled_true_sigma_arr = processed_dict['rescaled_true_sigma_arr']
llama_size = processed_dict['llama_size']
mode = processed_dict['mode']
refine_depth = processed_dict['refine_depth']
random_seed = processed_dict['random_seed']
PDF_list = processed_dict['PDF_list']
time_series = processed_dict['time_series']
prec = processed_dict['prec']
print(f"seed: {processed_dict['random_seed']}")
print(f"model name: {llama_size}")
print(f"mode: {mode}")
print(f"refine_depth: {refine_depth}")
print(f"prec: {prec}")
print(f"name: {file.stem}")

seed: 3
model name: histogram
mode: all
refine_depth: 1
prec: 2
name: uncorrelated_gaussian_centered_sigma_0.8_histogram_1


In [13]:
### tune temperature 

for PDF in PDF_list:
    if alpha != 1:
        PDF.rescale_temperature(alpha)

In [14]:
### Calculate true discretized distribution: PDF_list_true
### Applies to gaussian distribution only

PDF_true_list = copy.deepcopy(PDF_list)
for PDF, PDF_true, true_mean, true_sigma in zip(PDF_list, PDF_true_list, rescaled_true_mean_arr, rescaled_true_sigma_arr):
    def cdf(x):
        if dist_type == "gaussian":
            return 0.5 * (1 + erf((x - true_mean) / (true_sigma * np.sqrt(2))))
        
        elif dist_type == "uniform":
            lower_bound = true_mean - true_sigma
            upper_bound = true_mean + true_sigma
            return np.where(
                (x >= lower_bound) & (x <= upper_bound),
                (x - lower_bound) / (2 * true_sigma),
                np.where(x < lower_bound, 0, 1)
            )
        else:
            raise NotImplementedError("This distribution type is not implemented.")
    
    PDF_true.discretize(cdf, mode = "cdf")
    PDF_true.compute_stats()

In [15]:
from ipywidgets import interact
import matplotlib.pyplot as plt
from scipy.stats import norm, uniform
save_path = f"../figures/BM_example_temp{alpha}.png"
### Load multiple digits to MultiResolutionPDF
comma_locations = np.sort(np.where(np.array(list(full_series)) == ',')[0])
time_series_rescaled = (time_series-time_series.min()) / (time_series.max()-time_series.min()) * (8.5-1.5) + 1.5

plot1_log_scale = 0
log_scale = 0
truth_PDF = 0
# final_state = 709
final_state = None

### Plot distribution before ith comma
def digiprob_plotter(comma_idx=1):
    if comma_idx == 0:
        start_idx = 0
    else:
        start_idx = comma_locations[comma_idx-1]+1
    if truth_PDF:
        fig, axs = plt.subplots(3, 1, figsize=(8, 5), dpi = 200)
    else:
        fig, axs = plt.subplots(2, 1, figsize=(8, 10/3), dpi = 200)
    # Adjust the horizontal space between subplots
    plt.subplots_adjust(hspace=0.7)
    # Plot the full array with a marker on the selected value
    if final_state is not None:
        time_series_plot = time_series_rescaled[:final_state+1]
        axs[0].set_xlim(-10, final_state+10)
    else:
        time_series_plot = time_series_rescaled
    axs[0].plot(time_series_plot[:-1], marker='o', color='black', markersize=2, lw = '0.1', label = "observation")
    axs[0].scatter(comma_idx, time_series_plot[comma_idx], color='r', marker='o', label = "prediction")
    axs[0].set_xlabel('context lenght')
    axs[0].set_ylabel(r'rescaled $x$')
    
    if plot1_log_scale:
        axs[0].set_yscale('log')
        
    # Plot softmax distributions for each digit
    axs[1].set_ylabel("probability density")
    # axs[1].set_title(full_series[start_idx-30:start_idx] + "?")      
    PDF_list[comma_idx].compute_stats()
    PDF_list[comma_idx].plot(ax = axs[1], log_scale=log_scale, statistic = False)
    if truth_PDF:
        PDF_true_list[comma_idx].plot(ax = axs[2], log_scale=log_scale, statistic = False)
        axs[2].set_title('discretized truth')
        axs[2].set_xlabel("Digit")
    
    # characterizing ground truth distribution
    true_mean = rescaled_true_mean_arr[comma_idx]
    true_sigma = rescaled_true_sigma_arr[comma_idx]
    if true_sigma == 0:
        axs[1].vlines(true_mean, 0, np.max(PDF_list[comma_idx].bin_height_arr), color='r', label='Truth', lw = 3, alpha = 0.7)
    else:
        x_values = np.linspace(0, 10, 300)
        if dist_type == "gaussian":
            true_PDF = norm.pdf(x_values, true_mean, true_sigma)
        elif dist_type == "uniform": 
            true_PDF = uniform.pdf(x_values, true_mean-true_sigma, true_sigma*2)
        else:
            raise NotImplementedError("This distribution type is not implemented.")
        axs[1].plot(x_values, true_PDF, color = 'r', label = 'Truth', lw = 3, alpha = 0.7)
        # axs[1].vlines(true_mean, 0, np.max(gaussian), color = 'r')
    # axs[1].set_ylim(np.min(pdf.bin_height_arr), np.max(gaussian)*1.7)
    axs[1].set_xlabel(r'rescaled $x$')
    axs[1].bar([0], [0], color='dodgerblue', label='LLM Prediction')
    axs[1].set_xlim(0,10)
    # axs[1].set_ylim(0,5)
    axs[0].legend()
    axs[1].legend()
    if log_scale:
        axs[1].set_ylim(0.005,10)
        if truth_PDF:
            axs[2].set_ylim(0.0005,10)
    if final_state is not None: 
        plt.tight_layout()
        plt.savefig(save_path)

# Adjust the range of the slider to match the number of commas in the series
num_commas = full_series.count(',')
if final_state is not None: 
    interact(digiprob_plotter, comma_idx=final_state)
else:
    interact(digiprob_plotter, comma_idx=(1, num_commas-1, 1))

interactive(children=(IntSlider(value=1, description='comma_idx', max=399, min=1), Output()), _dom_classes=('w…

In [16]:
PDF_list[0].bin_center_arr

array([0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95, 1.05,
       1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95, 2.05, 2.15,
       2.25, 2.35, 2.45, 2.55, 2.65, 2.75, 2.85, 2.95, 3.05, 3.15, 3.25,
       3.35, 3.45, 3.55, 3.65, 3.75, 3.85, 3.95, 4.05, 4.15, 4.25, 4.35,
       4.45, 4.55, 4.65, 4.75, 4.85, 4.95, 5.05, 5.15, 5.25, 5.35, 5.45,
       5.55, 5.65, 5.75, 5.85, 5.95, 6.05, 6.15, 6.25, 6.35, 6.45, 6.55,
       6.65, 6.75, 6.85, 6.95, 7.05, 7.15, 7.25, 7.35, 7.45, 7.55, 7.65,
       7.75, 7.85, 7.95, 8.05, 8.15, 8.25, 8.35, 8.45, 8.55, 8.65, 8.75,
       8.85, 8.95, 9.05, 9.15, 9.25, 9.35, 9.45, 9.55, 9.65, 9.75, 9.85,
       9.95])

In [20]:
a = np.float64(2.0)  # double precision
b = np.float32(2.0)  # single precision
a==b

True